## Project: Building a database for crime reports


The goal of this guided project is to setup a database from scratch and put the Boston crime data into it.

The following user groups will be created:

readonly: Users in this group will have permission to read data only.
readwrite: Users in this group will have permissions to read and alter data but not to delete tables.

### Step 1: Creating the database and the schema
Creating a database named crime_db and a schema named crimes for storing the tables for containing the crime data.

In [1]:
import psycopg2
conn = psycopg2.connect(dbname = "dq", user = "dq")
## set autocommit to True bacause this is required for creating databases
conn.autocommit = True
cur = conn.cursor()
## create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()
import warnings
warnings.filterwarnings('ignore')

ProgrammingError: database "crime_db" already exists


In [2]:
## now that the crime_db database exists too we can connect to it
conn = psycopg2.connect(dbname = "crime_db", user = "dq")
conn.autocommit = True
cur = conn.cursor()
## create he crimes schema
cur.execute("CREATE SCHEMA crimes;")

ProgrammingError: schema "crimes" already exists


### Obtaining the Column Names and Sample
Obtaining the header row and assigning it to a variable named col_headers and obtaining the first data row and assigning it to a variable named first_row.

In [3]:
import csv

with open("boston.csv") as file:
    reader = list(csv.reader(file))
    col_headers = reader[0]
    first_row = reader[1]

print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


### Creating a function for analyzing column values
Creating a function get_col_set that given a CSV file name and a column index computes the set of all distinct values in that column.

The function will be used on each column to evaluate which columns have a lot of different values. Columns with a limited set of possible values are good candidates for enumerated datatypes.

In [4]:
def get_col_set(csv_file, index):
    import csv
    values = set()
    with open(csv_file, 'r') as file:
        next(file)
        reader = list(csv.reader(file))
        for row in reader:
            values.add(row[index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\n')

incident_number
298329
offense_code
219
description
239
date
1177
day_of_the_week
7
lat
18177
long
18177


### Analyzing the maximum length of the description column
Makig use of the get_col_set function to compute the maximum description length to decide an appropriate length for that field.

In [5]:
## inspect to see which index "description" is
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
description_unique = get_col_set("boston.csv", 2)

## METHOD 1
word = None
for words in description_unique:
    if word is None or len(words) > len(word):
        word = words
        
print(len(word))

## METHOD 2
max_len = 0
for words in description_unique:
    max_len = max(max_len, len(words))
print(max_len)

58
58


### Creating the table
An enumerated datatype, named weekday, for the day_of_the_week will be created since there is only seven possible values.

For the incident_number the type INTEGER will be used and will be set as the primary key. The same datatype will also used to represent the offense_code.

Since the description column has at most 58 character the datatype VARCHAR(100) will be used for representing it. This leaves some margin while not being so big that it will waste a lot of memory.

The date will use the DATE datatype. Finally, for the latitude and longitude the DECIMAL datatype will be used  in order to hold a good amount of precision.

In [7]:
print(col_headers)
print(first_row)


['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [8]:
# create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ProgrammingError: type "weekday" already exists


In [9]:
## create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ProgrammingError: relation "boston_crimes" already exists


### Load the data into the table
Making use of copy_expert to load the data as it is very fast and very succinct to use.

In [10]:
# load the data from boston.csv into the table boston_crimes that is in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")

IntegrityError: duplicate key value violates unique constraint "boston_crimes_pkey"
DETAIL:  Key (incident_number)=(1) already exists.
CONTEXT:  COPY boston_crimes, line 2


In [11]:
cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


### Revoking public privileges
Revoking all privileges of the public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

One also needs to revoke all privileges in the newly created schema. This is to ensure privileges are not granted by default.

In [12]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
cur.execute("REVOKE ALL ON SCHEMA crimes FROM public;")

### Creating the read only group
Creating a readonly group with NOLOGIN because it is a group and not a user. Now to grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then to deal wit tables privileges by granting SELECT. To add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new tables that are created on the crimes schema. As mentioned, by default no privileges are given. However this is changed so that by default any user in the readonly group can issue select commands.

In [13]:
cur.execute("CREATE GROUP readonly NOLOGIN;")

ProgrammingError: role "readonly" already exists


In [15]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")

In [16]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

In [14]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

### Creating the read-write group
Creating a readwrite group with NOLOGIN because it is a group and not a user. To grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then to deal with tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before one should change the default privileges so that a user in the readwrite group have these privileges if we ever create a new table on the crimes schema.

In [17]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")

ProgrammingError: role "readwrite" already exists


In [18]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

In [19]:
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

In [20]:
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

### Creating one user for each group
Creating a user named data_analyst with password secret1 in the readonly group.

Creating a user named data_scientist with password secret2 in the readwrite group.

In [21]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")


ProgrammingError: role "data_analyst" already exists


In [22]:
cur.execute("GRANT readonly TO data_analyst;")

In [23]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

ProgrammingError: role "data_scientist" already exists


In [24]:
cur.execute("GRANT readwrite TO data_scientist;")

### Test the database setup
Test the database setup using SQL queries on the pg_roles table and information_schema.table_privileges.

In the pg_roles table to inspect database related privileges and to look at the following columns:

rolname: The name of the user / group that the privilege refers to.
rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.
In the information_schema.table_privileges we will check privileges related to SQL queries on tables. Now to list the privileges of each group that was created earlier.

In [25]:
conn.close()
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
##check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print("\n")
## check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
